In [1]:
import folium
import os
import geopandas as gpd
import pandas as pd

# from geopy.point import Point
from shapely.geometry import Point
from shapely.geometry import box

In [2]:
filter_folder = 'shape-filter'
shapefiles = [file for file in os.listdir(filter_folder) if file.endswith('.shp')]
filter_gdf = [gpd.read_file(os.path.join(filter_folder, shp)) for shp in shapefiles]
filter_df = gpd.GeoDataFrame(pd.concat(filter_gdf, ignore_index=True))
filter_df

,AQDATE,FCODE,KLSRMP,KODATC,NAMOBJ,PUDATE,REMARK,SRS_ID,JNSPDG,LCODE,...,KRPPHN,KODLCO,KLSLCO,KRAPAT,SPSABS,RPTHTN,JNSHTB,TKTHTN,TIPHTN,JNSSMK
0,None,None,1.0,None,None,None,Padang Rumput,None,1.0,FB0020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,None,1.0,None,None,None,Padang Rumput,None,1.0,FB0020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,None,1.0,None,None,None,Padang Rumput,None,1.0,FB0020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,None,1.0,None,None,None,Padang Rumput,None,1.0,FB0020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,1.0,None,None,None,Padang Rumput,None,1.0,FB0020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,None,None,NaN,NaN,None,None,Semak Belukar,None,NaN,FB0040,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
715,None,None,NaN,NaN,None,None,Semak Belukar,None,NaN,FB0040,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
716,None,None,NaN,NaN,None,None,Semak Belukar,None,NaN,FB0040,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
717,None,None,NaN,NaN,None,None,Semak Belukar,None,NaN,FB0040,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [3]:
coordinates = pd.read_csv('archive/coordinates/filtered_land_points.csv')
coordinates = coordinates.filter(items=['lat', 'lon'])
coordinates

,lat,lon
0,-8.848272,115.128369
1,-8.848272,115.137460
2,-8.848272,115.146550
3,-8.848271,115.155641
4,-8.848271,115.164732
...,...,...
6401,-8.104389,115.294892
6402,-8.104389,115.303965
6403,-8.104389,115.313038
6404,-8.104389,115.322111


In [4]:
geometry = [Point(xy) for xy in zip(coordinates['lon'], coordinates['lat'])]
points_df = gpd.GeoDataFrame(coordinates, geometry=geometry)
points_df = points_df.set_crs(crs=filter_df.crs, allow_override=True)

In [5]:
points_inside = gpd.sjoin(points_df, filter_df, how='left', predicate='within')

In [6]:
points_outside = points_inside[points_inside['index_right'].isna()]
filtered_points = coordinates[coordinates.index.isin(points_outside.index)]

In [7]:
filtered_points

,lat,lon
0,-8.848272,115.128369
1,-8.848272,115.137460
2,-8.848272,115.146550
3,-8.848271,115.155641
4,-8.848271,115.164732
...,...,...
6401,-8.104389,115.294892
6402,-8.104389,115.303965
6403,-8.104389,115.313038
6404,-8.104389,115.322111


In [8]:
print(filtered_points.dtypes)

lat    float64
lon    float64
dtype: object


In [9]:
bounding_box = box(minx=114.192931152358, miny=-8.780735108712086, maxx=114.75836385136434, maxy=-8.440035251835656)

In [10]:
def box_check(lat, lon, filter_box):
    return not filter_box.contains(Point(lon, lat))

In [11]:
filtered_points = filtered_points[filtered_points.apply(lambda x: box_check(x['lat'], x['lon'], bounding_box), axis=1)]

In [12]:
filtered_points

,lat,lon
0,-8.848272,115.128369
1,-8.848272,115.137460
2,-8.848272,115.146550
3,-8.848271,115.155641
4,-8.848271,115.164732
...,...,...
6401,-8.104389,115.294892
6402,-8.104389,115.303965
6403,-8.104389,115.313038
6404,-8.104389,115.322111


In [13]:
forest_filtered = folium.Map(location=[filtered_points['lat'][0], filtered_points['lon'][0]], zoom_start=6)

for lat, lon in filtered_points.values:
    folium.Marker([lat, lon]).add_to(forest_filtered)

forest_filtered.save('map-export/forest_filtered_area.html')